In [1]:
import os
from PIL import Image
import torch
import torchvision
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
import torch.optim as optim
from utils import datasets
from utils import utils
from utils import train_val
from utils import net #网络文件于此
from utils import metrics
import warnings

# 完全禁用警告
warnings.filterwarnings("ignore")

In [2]:
from utils.utils import Config,Logs,BestSelector
config=utils.Config(
    dataset_sep=[
        0.82,0.17,0.01          
        ],
    resize_size=128,
    batch_size=128,
    lr=0.001,
    epochs=50,
    hidden_size=256,
    optim="Adam",
    momentum=0.9,
    weight_decay=1e-4,
    seed=42,
    mean= [0.50638 ,0.49962538 ,0.45205265],
    std=[0.23568255 ,0.24141274 ,0.25167742],
    source_dir=r"Cifar-10",#原始数据集，每个分类一个文件夹，每个文件夹里包含多个图片
    data_path=r"data\Cifar-10",#项目数据集
    # classes=["Apple","Carambola","Pear","Plum","Tomatoes"],
    device="cuda" if torch.cuda.is_available() else "cpu",
)
print(config)

dataset_sep : [0.82, 0.17, 0.01] 
resize_size : 128 
batch_size : 128 
lr : 0.001 
epochs : 50 
hidden_size : 256 
optim : Adam 
momentum : 0.9 
weight_decay : 0.0001 
seed : 42 
mean : [0.50638, 0.49962538, 0.45205265] 
std : [0.23568255, 0.24141274, 0.25167742] 
source_dir : Cifar-10 
data_path : data\Cifar-10 
device : cuda 


In [3]:
from tqdm import tqdm

#设置随机数种子
torch.manual_seed(config.seed)

isSplit= (not datasets.check_data_exist(config.data_path))#数据集不存在，则从原始数据存放处，转移数据集
if isSplit:
    print(f"{config.data_path} 数据集不存在，将从source_dir:{config.source_dir}中获取数据")
    print(f"清理源文件夹:{config.data_path}")
    datasets.clear_folder(config.data_path)
    for dir in tqdm(config.classes,desc="处理原始数据："):
        source_dir=os.path.join(config.source_dir, dir)
        print(source_dir)
        datasets.split_data(source_dir,target_dir=config.data_path,label=dir,sep=config.dataset_sep)
else:
    print(f"{config.data_path} 数据集已存在，无需重新划分")



data\Cifar-10 数据集已存在，无需重新划分


In [4]:
'''
def get_data_path(typ,path=config.data_path):
    return os.path.join(path,typ)

train_dataset=datasets.CustomImageDataset(
    get_data_path("train"),
    classes=config.classes,
    transform=datasets.get_transform(
        resize_size=config.resize_size,
        mean=config.mean,std=config.std
        )
)

val_dataset=datasets.CustomImageDataset(
    get_data_path("val"),
    classes=config.classes,
    transform=datasets.get_transform(
        "val",
        config.resize_size,
        
        mean=config.mean,
        std=config.std
        )
)
'''


'\ndef get_data_path(typ,path=config.data_path):\n    return os.path.join(path,typ)\n\ntrain_dataset=datasets.CustomImageDataset(\n    get_data_path("train"),\n    classes=config.classes,\n    transform=datasets.get_transform(\n        resize_size=config.resize_size,\n        mean=config.mean,std=config.std\n        )\n)\n\nval_dataset=datasets.CustomImageDataset(\n    get_data_path("val"),\n    classes=config.classes,\n    transform=datasets.get_transform(\n        "val",\n        config.resize_size,\n        \n        mean=config.mean,\n        std=config.std\n        )\n)\n'

#### 加载数据集，可本地数据集。调用datasets.CustomImageDataset

In [5]:
#加载cifar-10
import torchvision
train_val_dataset = torchvision.datasets.CIFAR10(
    root='./data/Cifar-10', 
    train=True, 
    download=True, 
    transform=datasets.get_transform(
        chance="train",
        resize_size=config.resize_size,
        mean=(0.5, 0.5, 0.5), 
        std=(0.5, 0.5, 0.5)
    )
)
train_dataset,val_dataset=datasets.get_parts_of_datasets(train_val_dataset,rate=0.75,only_train=False)#训练验证3-1开

test_dataset = torchvision.datasets.CIFAR10(root='./data/Cifar-10', train=False, download=True, transform=datasets.get_transform(
    chance="val",
    resize_size=config.resize_size,
    mean=(0.5, 0.5, 0.5), 
    std=(0.5, 0.5, 0.5)
))
print(f"原始训练集大小：{len(train_dataset)}")
print(f"原始验证集大小：{len(val_dataset)}")
print(f"原始测试集大小：{len(test_dataset)}")

Files already downloaded and verified
Files already downloaded and verified
原始训练集大小：37500
原始验证集大小：12500
原始测试集大小：10000


#### load数据集，并根据数据量进行裁剪

In [6]:
crop_rate=0.20
train_dataset_crop=datasets.get_parts_of_datasets(train_dataset,crop_rate)
val_dataset_crop=datasets.get_parts_of_datasets(val_dataset,crop_rate)
test_dataset_crop=datasets.get_parts_of_datasets(test_dataset,crop_rate)
train_dataset_size=len(train_dataset_crop)
val_dataset_size=len(val_dataset_crop)
test_dataset_size=len(test_dataset_crop)

print(f"本次训练用训练集大小：{len(train_dataset_crop)}")
print(f"本次训练用验证集大小：{len(val_dataset_crop)}")
print(f"本次测试用测试集大小：{len(test_dataset_crop)}")

train_loader=DataLoader(train_dataset_crop,batch_size=config.batch_size,shuffle=True,drop_last=True)
val_loader=DataLoader(val_dataset_crop,batch_size=config.batch_size,shuffle=False)
test_loader=DataLoader(test_dataset_crop,batch_size=config.batch_size,shuffle=False)

print(f"tarin_dataloader加载完毕, {len(train_loader)}个batch, batch大小为{config.batch_size}")
print(f"val_dataloader  加载完毕, {len(val_loader)}个batch, batch大小为{config.batch_size}")
print(f"test_dataloader 加载完毕, {len(test_loader)}个batch, batch大小为{config.batch_size}")


本次训练用训练集大小：7500
本次训练用验证集大小：2500
本次测试用测试集大小：2000
tarin_dataloader加载完毕, 58个batch, batch大小为128
val_dataloader  加载完毕, 20个batch, batch大小为128
test_dataloader 加载完毕, 16个batch, batch大小为128


#### 记录本次训练和测试用的数据量，还有数据样本信息

In [7]:
#查看数据加载情况
for inputs, labels in train_loader:
    inputs=inputs.to(config.device)
    labels=labels.to(config.device)
    print(inputs.shape)
    print(labels.shape)
    config.update(
        inputs_shape=inputs.shape
        )
    break
config.update(
    train_datasize=train_dataset_size,
    val_datasetsize=val_dataset_size,
    test_datasetsize=test_dataset_size,
    datasets_crop_rate=crop_rate,
    classes=train_val_dataset.classes,#原始数据集保留classes
)
config

torch.Size([128, 3, 128, 128])
torch.Size([128])


dataset_sep : [0.82, 0.17, 0.01] 
resize_size : 128 
batch_size : 128 
lr : 0.001 
epochs : 50 
hidden_size : 256 
optim : Adam 
momentum : 0.9 
weight_decay : 0.0001 
seed : 42 
mean : [0.50638, 0.49962538, 0.45205265] 
std : [0.23568255, 0.24141274, 0.25167742] 
source_dir : Cifar-10 
data_path : data\Cifar-10 
device : cuda 
inputs_shape : torch.Size([128, 3, 128, 128]) 
train_datasize : 7500 
val_datasetsize : 2500 
test_datasetsize : 2000 
datasets_crop_rate : 0.2 
classes : ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'] 

#### 加载预训练模型

In [8]:

def get_pretrained(hidden_size=config.hidden_size,config=config):
    '''
        获取预训练模型
        @param hidden_size: 隐藏层大小
        @param config: 配置文件
        @return: 预训练模型
    '''
    # model=net.BinaryClassificationMobileNetV3Large(out_size=len(config.classes))
    model=net.AutoCNN(
        input_channels=config.inputs_shape[1],
        input_size=config.inputs_shape[2:],
        num_classes=len(config.classes),
        mlp_hidden_size=256
    )
    return model.to(config.device)
test_model=get_pretrained()
inference_time=metrics.get_inference_time(test_model,config.device,input_shape=config.inputs_shape)
parameters_num=metrics.get_parameters_num(test_model)
print(f"inference_time:{inference_time} ms")
print(f"parameters_num:{parameters_num}")
config.update(
    network=test_model.__class__.__name__,
    inference_time=inference_time,
    parameters_num=parameters_num,

)

Testing ...: 100%|██████████| 300/300 [00:11<00:00, 27.12it/s]

inference_time:0.03631245431264241 ms
parameters_num:17151370


In [9]:

bestMod=utils.BestSelector(acc=0)
train_logs=utils.Logs()
model=get_pretrained()
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
# criterion = nn.BCELoss()
if config.optim == "Adam":
    optimizer = optim.Adam(model.parameters(), lr=config.lr)
else:
    optimizer = optim.SGD(model.parameters(), lr=config.lr, momentum=0.9, weight_decay=1e-4)
    
bestMod,train_logs=train_val.train_model(
            model, 
            criterion,
            optimizer, 
            train_loader,
            val_loader,
            bestMod=bestMod,
            train_logs=train_logs,
            config=config, 
            # num_epochs=config.epochs
        )



Training Epochs:   2%|▏         | 1/50 [00:16<13:43, 16.81s/it]

Epoch 1/50, Loss: 2.2205,  Val Accuracy: 0.2832, AP: 0.0793, Precision: 0.3195, Recall: 0.2832
当前最好的模型： acc : 0.2832 ,loss : 2.220451889366939 ,precision : 0.31950008017365195 ,recall : 0.28323044415649196 ,ap : 0.07927765262391004 ,epoch : 0 


Training Epochs:   4%|▍         | 2/50 [00:32<12:54, 16.14s/it]

Epoch 2/50, Loss: 2.1746,  Val Accuracy: 0.2800, AP: 0.0883, Precision: 0.3217, Recall: 0.2756
当前最好的模型： acc : 0.2832 ,loss : 2.220451889366939 ,precision : 0.31950008017365195 ,recall : 0.28323044415649196 ,ap : 0.07927765262391004 ,epoch : 0 


Training Epochs:   6%|▌         | 3/50 [00:48<12:30, 15.96s/it]

...

Epoch 48/50, Loss: 1.8717,  Val Accuracy: 0.5464, AP: 0.0890, Precision: 0.5521, Recall: 0.5477
当前最好的模型： acc : 0.552 ,loss : 1.887880645949265 ,precision : 0.5510116177740261 ,recall : 0.5520372635261089 ,ap : 0.08533942133983544 ,epoch : 44 


Training Epochs:  98%|█████████▊| 49/50 [12:44<00:15, 15.16s/it]

Epoch 49/50, Loss: 1.8730,  Val Accuracy: 0.5448, AP: 0.1018, Precision: 0.5503, Recall: 0.5450
当前最好的模型： acc : 0.552 ,loss : 1.887880645949265 ,precision : 0.5510116177740261 ,recall : 0.5520372635261089 ,ap : 0.08533942133983544 ,epoch : 44 


Training Epochs: 100%|██████████| 50/50 [12:59<00:00, 15.59s/it]

Epoch 50/50, Loss: 1.8774,  Val Accuracy: 0.5316, AP: 0.0917, Precision: 0.5385, Recall: 0.5329
当前最好的模型： acc : 0.552 ,loss : 1.887880645949265 ,precision : 0.5510116177740261 ,recall : 0.5520372635261089 ,ap : 0.08533942133983544 ,epoch : 44 


In [10]:
print(f"{config.epochs} epoch中 最好的模型")
print(bestMod)

50 epoch中 最好的模型
acc : 0.552 ,loss : 1.887880645949265 ,precision : 0.5510116177740261 ,recall : 0.5520372635261089 ,ap : 0.08533942133983544 ,epoch : 44 


#### 保存模型超参数和训练日志

In [11]:
saveDir=r'save_weights'
saveDir=os.path.join(saveDir,f'{bestMod.model.__class__.__name__}-acc={round(bestMod.acc,5)}-loss={round(bestMod.loss,6)}-max_epochs={config.epochs}-1100')
utils.saveProcess(
    saveDir=saveDir,
    bestMod=bestMod,
    train_log=train_logs,
    config=config
)

save_weights\AutoCNN-acc=0.552-loss=1.887881-max_epochs=50-1100


#### 进行测试

In [12]:
import torch
from utils import utils,train_val
import os

# config=Config(os.path.join(dir,'config.json'))
# model=BestSelector(os.path.join(dir,'metrics.json'))
saveDir=r'save_weights\BinaryClassificationMobileNetV3Large-acc=0.74336-loss=1.671334-max_epochs=40-1100'

Model,config,logs=utils.loadProcess(saveDir=saveDir)
metrics=train_val.validate_model(
    model=Model.model,
    val_loader=test_loader,
    device=config.device,
    only_val=True
)

metrics

模型测试中:: 100%|██████████| 16/16 [00:01<00:00,  8.03it/s]


{'Accuracy': 0.572,
 'Precision': 0.6234483008635754,
 'Recall': 0.5734097219219197,
 'F1': 0.5735498695026443,
 'AP': 0.06495701079185362}

#### 保存数据到tensorboard

In [13]:
from utils import metrics,utils


recoder=metrics.TensorboardRecorder(
    log_dir="runs/"
)

recoder.logs_scalars(
    logs.logs,
    prefix="train"
)



#### 多个模型的训练日志展示
 

In [14]:
from utils import metrics,utils
import os
model_dirs=[
    r"D:\Desktop\深度学习\12.康复训练\My_DL_Utils\Project\save_weights\AutoCNN-acc=0.3832-loss=2.082528-max_epochs=10-1100",
    r"D:\Desktop\深度学习\12.康复训练\My_DL_Utils\Project\save_weights\BinaryClassificationMobileNetV3Large-acc=0.662-loss=1.719393-max_epochs=10-1100"
]
recorders=[]
for model_dir in model_dirs:
    Model,config,logs=utils.loadProcess(model_dir)
    dirname=os.path.basename(model_dir)
    recorder=metrics.TensorboardRecorder(log_dir=f"runs/{dirname}")
    recorders.append(recorder)
    recorder.logs_scalars(
        scalars=logs.logs,
        prefix="train"
    )

